In [2]:
import pandas as pd
import numpy as np

train = pd.read_csv('hacktrain.csv')
test = pd.read_csv('hacktest.csv')

test
train

Unnamed: 0     ID       class  20150720_N  20150602_N  20150517_N  \
0              0      1       water    637.5950     658.668   -1882.030   
1              1      2       water    634.2400     593.705   -1625.790   
2              3      4       water     58.0174   -1599.160         NaN   
3              4      5       water     72.5180         NaN     380.436   
4              7      8       water   1136.4400         NaN         NaN   
...          ...    ...         ...         ...         ...         ...   
7995       10537  10538  impervious   1207.7000     984.620         NaN   
7996       10538  10539  impervious   2170.3500    1419.720    1361.000   
7997       10541  10542  impervious   1895.6800    1454.740         NaN   
7998       10542  10543  impervious   3465.7400    1283.320     413.412   
7999       10544  10545  impervious   6941.1900    1667.870    5084.780   

      20150501_N  20150415_N  20150330_N  20150314_N  ...  20140610_N  \
0       -1924.36     997.904   -1739.990     630.087  ...         NaN   
1       -1672.32     914.198    -692.386     707.626  ...         NaN   
2       -1052.63         NaN   -1564.630         NaN  ...   -1025.880   
3       -1256.93     515.805   -1413.180    -802.942  ...   -1813.950   
4        1647.83    1935.800         NaN    2158.980  ...    1535.000   
...          ...         ...         ...         ...  ...         ...   
7995     1166.25     937.478    1072.700     823.896  ...    1117.740   
7996     1478.71     983.911    1262.110    1422.860  ...     984.634   
7997     1033.56    1930.380    1057.150    1471.600  ...     888.408   
7998     4391.05    1146.820    4473.050    1614.750  ...    5833.760   
7999         NaN    1588.950    5978.190         NaN  ...    7352.570   

      20140525_N  20140509_N  20140423_N  20140407_N  20140322_N  20140218_N  \
0      -1043.160   -1942.490     267.138         NaN         NaN     211.328   
1       -933.934    -625.385     120.059     364.858     476.972     220.878   
2        368.622         NaN   -1227.800     304.621         NaN     369.214   
3        155.624         NaN    -924.073     432.150     282.833     298.320   
4       1959.430    -279.317    -384.915    -113.406    1020.720    1660.650   
...          ...         ...         ...         ...         ...         ...   
7995    1176.600    1044.110         NaN     369.082     465.843     362.882   
7996    2128.970    1379.660         NaN     762.633     485.204     446.724   
7997    2093.020    1232.110    1190.830    1441.460    1170.880    1095.000   
7998    4047.320    4515.800     433.177     277.296     744.143         NaN   
7999    3289.860    3729.150    1994.980         NaN    5299.900         NaN   

      20140202_N  20140117_N  20140101_N  
0      -2203.020    -1180.19     433.906  
1      -2250.000    -1360.56     524.075  
2      -2202.120         NaN   -1343.550  
3      -2197.360         NaN    -826.727  
4       -116.801     -568.05   -1357.140  
...          ...         ...         ...  
7995     979.795         NaN     433.659  
7996     771.747     1589.06     506.936  
7997    1818.650     2501.72    1247.770  
7998    3759.710         NaN     388.346  
7999    5983.130     1249.71    2424.230  

[8000 rows x 30 columns]

In [3]:
ndvi_cols = [col for col in train.columns if '_N' in col]


In [9]:
label_map = {'water': 0, 'impervious': 1, 'farm': 2, 'forest': 3, 'grass': 4, 'orchard': 5}
y_train = train['class'].map(label_map)

In [4]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='mean')
X_train_ndvi = imputer.fit_transform(train[ndvi_cols])
X_test_ndvi = imputer.transform(test[ndvi_cols])


In [11]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_ndvi)
X_test_scaled = scaler.transform(X_test_ndvi)


In [16]:
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(
    X_train_scaled, y_train, test_size=0.1, random_state=42, stratify=y_train
)

In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000)
model.fit(X_tr, y_tr)
val_preds = model.predict(X_val)
print("Validation Accuracy:", accuracy_score(y_val, val_preds))

D:\anaconda\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Validation Accuracy: 0.8725


In [22]:
test_preds = model.predict(X_test_scaled)
test_preds


array([2, 3, 5, ..., 0, 0, 1], dtype=int64)

# For the First method:
### I have predicted the hacktest.csv values which are named above as "test_preds". This doesnt have accuracy as the csv file (accd to me ) doesnt contain labels. Since ,it doesnt contain labels it is hard to validate the accuracy score. therefore i have taken the accuracy of previously trained dataset which is 87.2% 

In [24]:
def add_features(df):
    df_feat = df.copy()

    # Basic NDVI statistics
    df_feat['ndvi_mean'] = df[ndvi_cols].mean(axis=1)
    df_feat['ndvi_std'] = df[ndvi_cols].std(axis=1)
    df_feat['ndvi_max'] = df[ndvi_cols].max(axis=1)
    df_feat['ndvi_min'] = df[ndvi_cols].min(axis=1)
    df_feat['ndvi_range'] = df_feat['ndvi_max'] - df_feat['ndvi_min']
    df_feat['ndvi_median'] = df[ndvi_cols].median(axis=1)
    df_feat['ndvi_q1'] = df[ndvi_cols].quantile(0.25, axis=1)
    df_feat['ndvi_q3'] = df[ndvi_cols].quantile(0.75, axis=1)
    df_feat['ndvi_iqr'] = df_feat['ndvi_q3'] - df_feat['ndvi_q1']
    
    return df_feat

#have tried feature engineering which would increase my prediction accuracy, as we have more number of NVDI value columns it will be hard to predict the perfect accuracy as seen above hence i ve detailed it into the above 8 columns.

In [25]:
train_feat = add_features(train)
test_feat = add_features(test)

In [26]:
y_train = train['class']
X_train_raw = train_feat[ndvi_cols + [
    'ndvi_mean', 'ndvi_std', 'ndvi_max', 'ndvi_min',
    'ndvi_range', 'ndvi_median', 'ndvi_q1', 'ndvi_q3', 'ndvi_iqr'
]]

In [27]:
X_test_raw = test_feat[ndvi_cols + [
    'ndvi_mean', 'ndvi_std', 'ndvi_max', 'ndvi_min',
    'ndvi_range', 'ndvi_median', 'ndvi_q1', 'ndvi_q3', 'ndvi_iqr'
]]


In [28]:
imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train_raw)
X_test_imputed = imputer.transform(X_test_raw)

In [29]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_imputed)
X_test_scaled = scaler.transform(X_test_imputed)

In [32]:
X_tr, X_val, y_tr, y_val = train_test_split(
    X_train_scaled, y_train, test_size=0.2, random_state=42, stratify=y_train
)

model1 = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000)
model1.fit(X_tr, y_tr)

val_preds = model1.predict(X_val)

val_accuracy = accuracy_score(y_val, val_preds)
print("Validation Accuracy:", val_accuracy)

D:\anaconda\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Validation Accuracy: 0.905625


In [33]:
test_preds1 = model1.predict(X_test_scaled)
test_preds1

array(['orchard', 'orchard', 'orchard', ..., 'water', 'water', 'water'],
      dtype=object)